# Desenvolvimento do modelo de score de crédito

Exploração inicial de modelos de base

In [14]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, classification_report, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

## Carregando Dataset

In [15]:
ds = datasources.get_datasource("pedromonnt/fiap-credit-score-classification-model", "processed")

In [16]:
ds.all().dataframe

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,train-processed.csv,103365851,https://dagshub.com/api/v1/repos/pedromonnt/fi...,text/plain,16635276


In [17]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

Output()

In [18]:
dataset_url

'https://dagshub.com/api/v1/repos/pedromonnt/fiap-credit-score-classification-model/raw/main/data/processed/train-processed.csv'

In [19]:
df = pd.read_csv(dataset_url)
df.head()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_Desconhecido,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,...,0,1,0,0,0,0,1,0,0,0
1,23.0,19114.12,3093.745000,3.0,4.0,3.0,4.0,-1.0,14.0,11.27,...,0,1,0,0,0,0,0,1,0,0
2,-3.0,19114.12,3093.745000,3.0,4.0,3.0,4.0,3.0,7.0,9.40,...,0,1,0,0,0,0,0,0,1,0
3,23.0,19114.12,3093.745000,3.0,4.0,3.0,4.0,5.0,4.0,6.27,...,0,1,0,0,0,0,0,0,0,1
4,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,6.0,14.0,11.27,...,0,1,0,0,0,1,0,0,0,0


## Desenvolvimento e experimentos de modelos

In [20]:
dagshub.init(repo_owner="pedromonnt", repo_name="fiap-credit-score-classification-model", mlflow=True)

Initialized MLflow to track repo "pedromonnt/fiap-credit-score-classification-model"

Repository pedromonnt/fiap-credit-score-classification-model initialized!

In [21]:
mlflow.autolog()

2025/07/17 22:15:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/17 22:15:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/17 22:15:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [22]:
# Features e Target do conjunto de treino
y = df['Credit_Score']
X = df.drop(columns='Credit_Score')

# Dividir o conjunto de TREINO em treino e VALIDAÇÃO
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [23]:
def evaluate_and_log_model(kind, model_name, model, X_val, y_val):
    predictions = model.predict(X_val)
    pred_proba = model.predict_proba(X_val)

    # Métricas
    accuracy = accuracy_score(y_val, predictions)
    f1 = f1_score(y_val, predictions, average='weighted')
    precision = precision_score(y_val, predictions, average='weighted')
    recall = recall_score(y_val, predictions, average='weighted')
    roc_auc = roc_auc_score(y_val, pred_proba, multi_class='ovr', average='weighted')

    print(f"Resultados para {model_name}:")
    print(f"  Acurácia: {accuracy:.4f}")
    print(f"  F1-Score (Weighted): {f1:.4f}")
    print(f"  ROC AUC (Weighted): {roc_auc:.4f}")

    # Log de Métricas no MLflow
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_weighted", f1)
    mlflow.log_metric("precision_weighted", precision)
    mlflow.log_metric("recall_weighted", recall)
    mlflow.log_metric("roc_auc_weighted", roc_auc)

    signature = infer_signature(X_val, predictions)

    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_val[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_val[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_val[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_val[:5])

    print(f"Model {model_name} logged with accuracy: {accuracy}, f1_weighted: {f1}, precision_weighted: {precision}, recall_weighted: {recall}, roc_auc_weighted: {roc_auc}")

    # Log do Relatório de Classificação como artefato de texto
    report = classification_report(y_val, predictions)
    mlflow.log_text(report, "classification_report.txt")

    # Log do modelo
    mlflow.sklearn.log_model(model, model_name)

### Modelo 1: Regressão Logística

In [ ]:
with mlflow.start_run(run_name="Logistic Regression"):
   
    # Parâmetros para GridSearchCV
    param_grid_lr = {
        'C': [0.1, 1.0, 10.0],
        'solver': ['liblinear', 'saga']
    }
    
    lr = LogisticRegression(random_state=42, max_iter=1000)

    grid_search_lr = GridSearchCV(lr, param_grid_lr, cv=5, scoring='f1_weighted', n_jobs=-1)
    grid_search_lr.fit(X_train, y_train)
    
    best_model_lr = grid_search_lr.best_estimator_
    mlflow.log_params(grid_search_lr.best_params_)
    
    evaluate_and_log_model("sklearn", "logistic_regression", best_model_lr, X_val, y_val)

2025/07/17 22:15:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/17 22:24:47 WARNING mlflow.utils.autologging_u

🏃 View run rumbling-jay-424 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/d230974a3fd44c428684444ad9ff652d
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0
🏃 View run omniscient-shark-973 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/e7d4dffaaafb4ff899150ea0d159ca5f
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0
🏃 View run unique-foal-487 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/ab3bd22a518f4e99a330e73c2f32d474
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0
🏃 View run illustrious-toad-920 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/61c55dc651c847ba9cffb15595e52

c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Model logistic_regression logged with accuracy: 0.6288, f1_weighted: 0.6213540988999845, precision_weighted: 0.6245405785213104, recall_weighted: 0.6288, roc_auc_weighted: 0.7693549318172382


2025/07/17 22:25:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/df46dbd0d50e4942a4d4349071a5c338
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0


🏃 View run sedate-conch-674 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/2223b970f6684eef9c75fa67a641c2bb
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0
🏃 View run loud-shark-799 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/4e91bf93ced2431ca1762583e21c9fea
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0
🏃 View run stylish-dove-99 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/f95d0c4ae62142429057834feba530c2
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0
🏃 View run resilient-shad-188 at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/56a495f635544079a8569dc3f28414cd
🧪 Vi

### Modelo 2: Árvore de Decisão

In [25]:
with mlflow.start_run(run_name="Decision Tree Classifier"):
    dt = DecisionTreeClassifier(random_state=42)
    
    param_grid_dt = {
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='f1_weighted', n_jobs=-1)
    grid_search_dt.fit(X_train, y_train)

    best_model_dt = grid_search_dt.best_estimator_
    mlflow.log_params(grid_search_dt.best_params_)

    evaluate_and_log_model("sklearn", "decision_tree_classifier", best_model_dt, X_val, y_val)

2025/07/17 22:26:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/17 22:27:34 WARNING mlflow.utils.autologging_u

Resultados para decision_tree_classifier:
  Acurácia: 0.7088
  F1-Score (Weighted): 0.7110
  ROC AUC (Weighted): 0.8311


c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Model decision_tree_classifier logged with accuracy: 0.70885, f1_weighted: 0.7110410036315017, precision_weighted: 0.7159983852334575, recall_weighted: 0.70885, roc_auc_weighted: 0.8311445429371924


2025/07/17 22:28:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Decision Tree Classifier at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/d7912ddd228a40e996e5c4ffe7934d5e
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0


### Modelo 3: XGBoost Classifier

In [26]:
with mlflow.start_run(run_name="XGBoost Classifier"):
    
    xgb = XGBClassifier(random_state=42, eval_metric='mlogloss')
    
    param_grid_xgb = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0]
    }
    
    grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=5, scoring='f1_weighted', n_jobs=-1)
    grid_search_xgb.fit(X_train, y_train)

    best_model_xgb = grid_search_xgb.best_estimator_
    mlflow.log_params(grid_search_xgb.best_params_)
    
    evaluate_and_log_model("xgboost", "xgboost_classifier", best_model_xgb, X_val, y_val)

2025/07/17 22:29:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/17 22:34:50 WARNING mlflow.utils.autologging_u

Resultados para xgboost_classifier:
  Acurácia: 0.7620
  F1-Score (Weighted): 0.7618
  ROC AUC (Weighted): 0.8782


c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\fiap-credit-score-classification\fiap-credit-score-classification-model\.venv\Lib\site-pac

Model xgboost_classifier logged with accuracy: 0.76195, f1_weighted: 0.7617952702484266, precision_weighted: 0.7618689695604779, recall_weighted: 0.76195, roc_auc_weighted: 0.8781685603706157


2025/07/17 22:35:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost Classifier at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0/runs/a20e075a6f7a434ea7ebdf2bee2d585d
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-credit-score-classification-model.mlflow/#/experiments/0


## Registro de Modelo em Produção

In [30]:
run_id = "a20e075a6f7a434ea7ebdf2bee2d585d"

mlflow.register_model(model_uri=f"runs:/{run_id}/model", name="credit-score-classification-model")

Registered model 'credit-score-classification-model' already exists. Creating a new version of this model...
2025/07/17 22:43:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: credit-score-classification-model, version 10
Created version '10' of model 'credit-score-classification-model'.


<ModelVersion: aliases=[], creation_timestamp=1752802990112, current_stage='None', description='', last_updated_timestamp=1752802990112, name='credit-score-classification-model', run_id='a20e075a6f7a434ea7ebdf2bee2d585d', run_link='', source='mlflow-artifacts:/5cb5e553364e438896ea46ed8538561f/a20e075a6f7a434ea7ebdf2bee2d585d/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='10'>